Y-DATA 2020/21<br/>
NLP HW3<br/>
Serge Tochilov

# Transformers, BERT, Adapters

The goals of the present exercise are:
- Get acquainted with the `transformers` package, as well as with implementation of the BERT model
- Implement Adapters for transfer learning
- Get familiar with the GLUE benchmark

In [ ]:
!pip3 install -q datasets datasets transformers

## Problems with fine-tuning the entire model

As we learned in the NLP lecture #4, fine-tuning large pretrained models is the most effective transfer learning approach. However, if we have a lot of downstream tasks this method can be cumbersome, since for each new task a new model needs to be trained from scratch.

## Adapters: Parameter-efficient approach
In the present exercise we will implement another method for transfer learning, which only adds few trainable parameters per task, and new tasks can be added without revisiting the previous ones.

The method was proposed at https://arxiv.org/pdf/1902.00751.pdf, and the details regarding the implementation can be found in section 2.1 (for tuning the parameters you may also look at section 3).

*Please note that in addition to the adapter layers the authors also trained a new layer normalization, but for the present exercise it is not mandatory.*

> Let's start with implementation of the adapter layer.

In [ ]:
import numpy as np
import random
import pandas as pd
from IPython.display import display, HTML

import torch.nn as nn
import torch.nn.functional as F
import datasets
from datasets import load_dataset, load_metric


GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "cola"
model_checkpoint = "bert-base-uncased"
batch_size = 16

> The next cell is the paper's Adapters implementation written in `Tensorflow`. Some highlights:

> - Weights are initialized by normal distribution truncated to $\pm2\sigma$ with $\sigma=0.001$
- Biases are zero-initialized
- Non-linearity is GeLU
- Skip-forward connection is realized by adding an input to output of the upstream feed-forward layer

In [ ]:
def feedforward_adapter(input_tensor, hidden_size=64, init_scale=1e-3):
  """A feedforward adapter layer with a bottleneck.
  Implements a bottleneck layer with a user-specified nonlinearity and an
  identity residual connection. All variables created are added to the
  "adapters" collection.
  Args:
    input_tensor: input Tensor of shape [batch size, hidden dimension]
    hidden_size: dimension of the bottleneck layer.
    init_scale: Scale of the initialization distribution used for weights.
  Returns:
    Tensor of the same shape as x.
  """
  with tf.variable_scope("adapters"):
    in_size = input_tensor.get_shape().as_list()[1]
    w1 = tf.get_variable(
        "weights1", [in_size, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=init_scale),
        collections=["adapters", tf.GraphKeys.GLOBAL_VARIABLES])
    b1 = tf.get_variable(
        "biases1", [1, hidden_size],
        initializer=tf.zeros_initializer(),
        collections=["adapters", tf.GraphKeys.GLOBAL_VARIABLES])
    net = tf.tensordot(input_tensor, w1, [[1], [0]]) + b1

    net = gelu(net)

    w2 = tf.get_variable(
        "weights2", [hidden_size, in_size],
        initializer=tf.truncated_normal_initializer(stddev=init_scale),
        collections=["adapters", tf.GraphKeys.GLOBAL_VARIABLES])
    b2 = tf.get_variable(
        "biases2", [1, in_size],
        initializer=tf.zeros_initializer(),
        collections=["adapters", tf.GraphKeys.GLOBAL_VARIABLES])
    net = tf.tensordot(net, w2, [[1], [0]]) + b2

  return net + input_tensor

> We transcoded the adapter architecture from Tensorflow to PyTorch. Note that the code became much more concise and expressive. Hope that PyTorch will also have built-in `init.truncated_normal_` soon.

In [ ]:
def truncated_normal_(tensor, mean=0, std=1):
    tmp = tensor.new_empty(tensor.shape + (4,)).normal_()
    valid = (tmp < 2) & (tmp > -2)
    ind = valid.max(-1, keepdim=True)[1]
    tensor.data.copy_(tmp.gather(-1, ind).squeeze(-1))
    tensor.data.mul_(std).add_(mean)

In [ ]:
class Adapter(nn.Module):

    def __init__(self, input_size, hidden_size=64, init_scale=1e-3):

        super().__init__()

        self.ff_down = nn.Linear(input_size, hidden_size)
        truncated_normal_(self.ff_down.weight, std=init_scale)
        nn.init.zeros_(self.ff_down.bias)

        self.ff_up = nn.Linear(hidden_size, input_size)
        truncated_normal_(self.ff_up.weight, std=init_scale)
        nn.init.zeros_(self.ff_up.bias)


    def forward(self, x):

        x_c = self.ff_down(x)
        x_c = F.gelu(x_c)
        x_c = self.ff_up(x_c)
        return x_c + x

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task == "stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BERT (base) contains 12 transformer layers. Each layer has the following parameters:

*   attention.self.query 
*   attention.self.key 
*   attention.self.value 
*   attention.output.dense 
*   attention.output.LayerNorm 
*   intermediate.dense
*   output.dense 
*   output.LayerNorm

TIP: to iterate over all the layers and to see their corresponding sizes you can run the following code:

    for name, param in model.named_parameters():
        print(name, param.size())

> It's sufficient to check the first of $12$ identical layers.

In [ ]:
model.bert.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

Your need to find where to insert two additional adapter layers. After finding the appropriate location replace the placeholders `<place1>` and `<place2>` with the layer name that should come **after** the adapter. The next cell will insert to the model the adapters you implemented.

For example, if you want to place the adapter after `query` and `key` layers, run this code:

    adapters_dict = dict()
    for i in range(12):
        adapters_dict[i] = [Adapter(768), Adapter(768)]
        model.bert.encoder.layer[i].attention.self.query = nn.Sequential(adapters_dict[i][0], model.bert.encoder.layer[i].attention.self.query)
        model.bert.encoder.layer[i].attention.self.key = nn.Sequential(adapters_dict[i][1], model.bert.encoder.layer[i].attention.self.key)

> The insert position is before the layer normalizations for both downstream and upstream adapters. Input size in that position is $768$. We know the adapter names so no need to store them separately.

In [ ]:
for i in range(12):
    model.bert.encoder.layer[i].attention.output.LayerNorm = nn.Sequential(Adapter(768), model.bert.encoder.layer[i].attention.output.LayerNorm)
    model.bert.encoder.layer[i].output.LayerNorm = nn.Sequential(Adapter(768), model.bert.encoder.layer[i].output.LayerNorm)


In [ ]:
model.bert.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): Sequential(
        (0): Adapter(
          (ff_down): Linear(in_features=768, out_features=64, bias=True)
          (ff_up): Linear(in_features=64, out_features=768, bias=True)
        )
        (1): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): Sequential(
      (0)

The next thing that we need to do is to freeze all the layers except the one you changed above, and the classification head.

In [ ]:
for name, param in model.named_parameters():
    if not ("classifier." in name or ".ff_" in name):
        print("freezing", name)
        param.requires_grad = False

freezing bert.embeddings.word_embeddings.weight
freezing bert.embeddings.position_embeddings.weight
freezing bert.embeddings.token_type_embeddings.weight
freezing bert.embeddings.LayerNorm.weight
freezing bert.embeddings.LayerNorm.bias
freezing bert.encoder.layer.0.attention.self.query.weight
freezing bert.encoder.layer.0.attention.self.query.bias
freezing bert.encoder.layer.0.attention.self.key.weight
freezing bert.encoder.layer.0.attention.self.key.bias
freezing bert.encoder.layer.0.attention.self.value.weight
freezing bert.encoder.layer.0.attention.self.value.bias
freezing bert.encoder.layer.0.attention.output.dense.weight
freezing bert.encoder.layer.0.attention.output.dense.bias
freezing bert.encoder.layer.0.attention.output.LayerNorm.1.weight
freezing bert.encoder.layer.0.attention.output.LayerNorm.1.bias
freezing bert.encoder.layer.0.intermediate.dense.weight
freezing bert.encoder.layer.0.intermediate.dense.bias
freezing bert.encoder.layer.0.output.dense.weight
freezing bert.enco

# Fine-tuning on GLUE

After augmenting the BERT model with adapters, run the following cells.

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (/home/ubuntu/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': -0.0629940788348712}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used during pretraining this specific checkpoint.

The vocabulary is cached, so it won't be downloaded again next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error from the previous call, remove that argument.

You can directly call this tokenizer on one sentence, or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just remember they are required by the model that we instantiate later). You can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of correspondence between the task and column names:

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double-check if it does work on our current dataset:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


We can now write a function that will preprocess our samples. We just feed them to the `tokenizer` with the `truncation=True` argument. This will ensure that an input longer that the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To apply this function to all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function to all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7d85522739442b44.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1ab8abc8f4c213f4.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-0f0825e91600bad5.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus it needs to drop the cache data). For instance, it will properly detect if you changed the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, and you can use `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always $2$, except for STS-B which is a regression problem and MNLI where we have $3$ labels):

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head, for which we don't have pretrained weights, so the library warns us that we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook, and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the `argmax()` of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now fine-tune our model by just calling the `train` method for the CoLA dataset:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.583200,0.530612,0.367276
2,0.488900,0.485126,0.460998
3,0.464300,0.493656,0.469197
4,0.445400,0.478455,0.496613
5,0.443400,0.489801,0.474714


TrainOutput(global_step=2675, training_loss=0.4819134407399971, metrics={'train_runtime': 407.0419, 'train_samples_per_second': 6.572, 'total_flos': 0, 'epoch': 5.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it wasn't the last one):

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4784548282623291,
 'eval_matthews_correlation': 0.4966130731383176,
 'eval_runtime': 3.7013,
 'eval_samples_per_second': 281.792,
 'epoch': 5.0}

> The pretrained BERT + Adapters model reached a reasonable performance after just $4$ epochs. Let's now compare it with the fully-tuned pretrained BERT.

In [ ]:
model_no_adapters = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

trainer_no_adapters = Trainer(
    model_no_adapters,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer_no_adapters.train()
trainer_no_adapters.evaluate()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.491000,0.542611,0.486810
2,0.290800,0.615980,0.510463
3,0.186900,0.742682,0.556015
4,0.129500,0.867175,0.565789
5,0.085800,0.957321,0.555809


{'eval_loss': 0.8671747446060181,
 'eval_matthews_correlation': 0.5657894359264041,
 'eval_runtime': 3.535,
 'eval_samples_per_second': 295.053,
 'epoch': 5.0}

> The fully-tuned pretrained BERT performs better but heavily overfits.